## Project Initialization

In [ ]:
# Import Necessary Libraries
import sagemaker
import boto3 # Allow users to use AWS services like S3 and EC2
from sagemaker.amazon.amazon_estimator import get_image_uri # Allow users to use SageMaker built-in algorithms
from sagemaker.image_uris import retrieve
from sagemaker.session import s3_input, Session

In [67]:
# Chekc the region of the Notebook instance
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [68]:
# Create the S3 bucket in the same region
bucket_name = 'bankapplication20201228' 

s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 bucket creation error: ', e)
    
    

S3 bucket created successfully


In [69]:
# Set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication20201228/xgboost-as-a-built-in-algo/output


## Data Pre-Processing

In [70]:
# Download The Dataset and Store it in S3
import pandas as pd
import urllib # URL handling module in Python
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv",
                              "bank_clean.csv") # Download the dataset in the Notebook Instance
    print('Success: download bank_clean.csv') 
except Exception as e:
    print('Data download error: ', e)

# Read the csv file and store it as a pandas data frame
model_data = pd.read_csv('./bank_clean.csv', index_col = 0)

Success: download bank_clean.csv


In [71]:
# Check the basic information about the columns in the dataset
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   age                            41188 non-null  int64
 1   campaign                       41188 non-null  int64
 2   pdays                          41188 non-null  int64
 3   previous                       41188 non-null  int64
 4   no_previous_contact            41188 non-null  int64
 5   not_working                    41188 non-null  int64
 6   job_admin.                     41188 non-null  int64
 7   job_blue-collar                41188 non-null  int64
 8   job_entrepreneur               41188 non-null  int64
 9   job_housemaid                  41188 non-null  int64
 10  job_management                 41188 non-null  int64
 11  job_retired                    41188 non-null  int64
 12  job_self-employed              41188 non-null  int64
 13  job_services    

In [72]:
# Train/Test Split
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state = 999), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [73]:
# Save traininng dataset into S3 bucket
import os
# Notice the single bracket here of selecting the 'y_yes' column, which produces a series in Pandas
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], 
          axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [74]:
# Save testing dataset into S3 bucket
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], 
          axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

In [75]:
# Read the training dataset and test dataset from S3 bucket
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
s3_input_test = sagemaker.TrainingInput(s3_data ='s3://{}/{}/test'.format(bucket_name,prefix),content_type='csv')
print(type(s3_input_train))
print(type(s3_input_test))

<class 'sagemaker.inputs.TrainingInput'>
<class 'sagemaker.inputs.TrainingInput'>


## Model Training

In [76]:
# Search for the XGBoost Training Images from the Elastic Container Repository
# Use :1 version to ensure the stable version
container = retrieve(framework = 'xgboost',region = my_region, version = '1.0-1')

In [77]:
# Initialize hyperparameters
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":50
}

In [78]:
# Construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [79]:
# Start the model training
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

2020-12-30 06:54:31 Starting - Starting the training job...
2020-12-30 06:54:54 Starting - Launching requested ML instancesProfilerReport-1609311270: InProgress
......
2020-12-30 06:56:01 Starting - Preparing the instances for training.........
2020-12-30 06:57:16 Downloading - Downloading input data
2020-12-30 06:57:16 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[06:57:38] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delim

## Model Deployment

In [80]:
# Deploy the model using SageMaker Hosting Service
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-----------------!

In [82]:
print(type(xgb_predictor))

<class 'sagemaker.predictor.Predictor'>


In [83]:
# Prediction of the test data using the deployed model
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [84]:
predictions_array

array([0.09626938, 0.0393134 , 0.10125767, ..., 0.05990799, 0.0801229 ,
       0.31911257])

In [85]:
# Generate the confusion matrix for the test dataset result
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10795)    37% (172)
Purchase        9% (1100)     63% (290) 



## Clean Up

In [ ]:
# Delete the endpoints and clear the objects in the S3 bucket
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint_name)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()